Read Mesh

In [1]:
from nassu.lnas import LagrangianFormat
import pathlib

mesh = LagrangianFormat.from_file(pathlib.Path("./output/simulation_test/lnas/G200/G200.transformed.lnas"))

Read historic series

In [2]:
import pandas as pd

data_path = pathlib.Path("./output/simulation_test/probes/hist_series/series1")

body_hist_series = pd.read_hdf(data_path / "bodies.G200.data.h5")
sfc1_data = pd.read_hdf(data_path / "bodies.m1_inferior.data.h5")
sfc2_data = pd.read_hdf(data_path / "bodies.m2_inferior.data.h5")
sfc3_data = pd.read_hdf(data_path / "bodies.m3_inferior.data.h5")

last_index = body_hist_series.point_idx.max()
sfc1_data.point_idx += last_index + 1

last_index = sfc1_data.point_idx.max()
sfc2_data.point_idx += last_index + 1

last_index = sfc2_data.point_idx.max()
sfc3_data.point_idx += last_index + 1

len(body_hist_series.index) + len(sfc1_data.index) + len(sfc2_data.index) + len(sfc3_data.index)

174184

Offset surface

In [3]:
import numpy as np

NORMAL_OFFSET = - 0.5

for surface_name in ["m1_main", "m2_main", "m3_main"]:
    tri_normals = mesh.geometry_from_surface(surface_name).normals.copy()
    tri_verts = mesh.geometry_from_surface(surface_name).triangle_vertices.copy()
    tri_verts += np.repeat(tri_normals, 3).reshape(-1,3,3) * NORMAL_OFFSET

    new_verts = tri_verts.copy().reshape(-1, 3)
    new_triangles = []

    for tri in tri_verts:
        tri_index = []
        for v in tri:
            matches = np.all(new_verts == v, axis=1)
            indices = np.where(matches)
            tri_index.append(indices[0][0])
        new_triangles.append(np.array(tri_index))

    new_triangles = np.array(new_triangles) + len(mesh.geometry.vertices)

    mesh.geometry.vertices = np.concatenate((mesh.geometry.vertices, new_verts))
    mesh.geometry.triangles = np.concatenate((mesh.geometry.triangles, new_triangles))

    mesh.geometry._update_triangles_vertices()
    mesh.geometry._update_normals()
    mesh.geometry._update_areas()
    
filename = pathlib.Path("./output/simulation_test/lnas/G200/G200.merged.lnas")
mesh.to_file(filename)

Join historic series

In [4]:
new_df = pd.concat([body_hist_series, sfc1_data, sfc2_data, sfc3_data]).sort_values(by=["time_step", "point_idx"])
new_df.to_hdf(data_path / "bodies.G200.merged.data.h5", key='df', mode='w')